Датасет ml-latest. Вспомнить подходы, которые мы разбирали. Выбрать понравившийся подход к гибридным системам. Написать свою. Материалы вы найдёте здесь.https://netology.ru/profile/program/rsml-7/lessons/247787/lesson_items/1317244#:~:text=%D0%94%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20ml%2Dlatest,%D0%B2%D1%8B%20%D0%BD%D0%B0%D0%B9%D0%B4%D1%91%D1%82%D0%B5%20%D0%B7%D0%B4%D0%B5%D1%81%D1%8C.

<font color=green> За основу возьмем готовую модель предсказания потенциального рейтинга клиента из первой домашней работы.

<font color=green> Встроим эту модель в предполагаемый сценарий работы, дополнив отсутствующими элементами.

<font color=green> Определим сценарий - клиент заходит в каталоге в конкретный жанр и ему выдается:
<br>
1 список из 7 самых популярных фильмов жанра на основании рейтингов фильмов
<br>
2 список из 7 самых популярных фильмов полученных после добавления к текущему жанру трех самых популярных жанра из истории клиента, и поиска по их комбинации методом ближайших соседей похожих фильмов с исключением уже просмотренных и рекомендованных по 1 списку и отсортированных по потенциальному рейтингу, полученному по модели случайного леса ( если истории еще нет, 3 дополнительных жанра выбираются случайным образом ).

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random


In [ ]:
!pip install icecream

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from icecream import ic

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2023-03-15 19:29:35--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 142.251.2.100, 142.251.2.101, 142.251.2.138, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/54uo4ub9kfvf3t6l10k9ditghsipvptv/1678908525000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=93697d17-7803-4e86-937d-954e8d176220 [following]
--2023-03-15 19:29:36--  https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/54uo4ub9kfvf3t6l10k9ditghsipvptv/1678908525000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=93697d17-7803-4e86-937d-954e8d176220
Resolving doc-0g-84-docs.googleusercontent.com (doc-0g-84-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to doc-0g-84

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [ ]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [ ]:
movies = movies.drop_duplicates()
movies.shape

(9742, 3)

In [ ]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [ ]:
len(movies.movieId.unique())

9742

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [ ]:
ratings = ratings.drop_duplicates()
ratings.shape

(100836, 4)

In [ ]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [ ]:
len(ratings.userId.unique())

610

In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [ ]:
tags = tags.drop_duplicates()
tags.shape

(3683, 4)

In [ ]:
len(tags.userId.unique())

58

<font color=green> Посчитаем средние рейтинги по пользователю и по фильму и сохраним результаты.

In [ ]:
ratings_u_mean = pd.DataFrame(ratings.groupby("userId")['rating'].mean())
ratings_u_mean = ratings_u_mean.reset_index().rename(columns={'rating': 'u_mean'})
ratings_u_mean.head()


,userId,u_mean
0,1,4.366379
1,2,3.948276
2,3,2.435897
3,4,3.555556
4,5,3.636364


In [ ]:
ratings_mean = ratings.merge(ratings_u_mean, how = 'inner', left_on='userId', right_on='userId')

In [ ]:
ratings_mean

,userId,movieId,rating,timestamp,u_mean
0,1,1,4.0,964982703,4.366379
1,1,3,4.0,964981247,4.366379
2,1,6,4.0,964982224,4.366379
3,1,47,5.0,964983815,4.366379
4,1,50,5.0,964982931,4.366379
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,3.688556
100832,610,168248,5.0,1493850091,3.688556
100833,610,168250,5.0,1494273047,3.688556
100834,610,168252,5.0,1493846352,3.688556


In [ ]:
ratings_m_mean = pd.DataFrame(ratings.groupby("movieId")['rating'].mean())
ratings_m_mean = ratings_m_mean.reset_index().rename(columns={'rating': 'm_mean'})
ratings_m_mean.head()


,movieId,m_mean
0,1,3.920930
1,2,3.431818
2,3,3.259615
3,4,2.357143
4,5,3.071429


In [ ]:
ratings_mean = ratings_mean.merge(ratings_m_mean, how = 'inner', left_on='movieId', right_on='movieId')


In [ ]:
ratings_mean

,userId,movieId,rating,timestamp,u_mean,m_mean
0,1,1,4.0,964982703,4.366379,3.92093
1,5,1,4.0,847434962,3.636364,3.92093
2,7,1,4.5,1106635946,3.230263,3.92093
3,15,1,2.5,1510577970,3.448148,3.92093
4,17,1,4.5,1305696483,4.209524,3.92093
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,3.688556,2.50000
100832,610,160527,4.5,1479544998,3.688556,4.50000
100833,610,160836,3.0,1493844794,3.688556,3.00000
100834,610,163937,3.5,1493848789,3.688556,3.50000


<font color=green> Нормируем оценки по пользователю и по фильму к среднему значению, для более "нормального" распределения.<br>
Оценки по модулю ограничены и к тому же, для предсказания оценки важнее участия в отдельной категории, поэтому к базе от 0 до 1 приводить их не будем.

In [ ]:
ratings_mean['r_u'] =ratings_mean['rating']-ratings_mean['u_mean']
ratings_mean['m_u'] =ratings_mean['rating']-ratings_mean['m_mean']
ratings_mean

,userId,movieId,rating,timestamp,u_mean,m_mean,r_u,m_u
0,1,1,4.0,964982703,4.366379,3.92093,-0.366379,0.07907
1,5,1,4.0,847434962,3.636364,3.92093,0.363636,0.07907
2,7,1,4.5,1106635946,3.230263,3.92093,1.269737,0.57907
3,15,1,2.5,1510577970,3.448148,3.92093,-0.948148,-1.42093
4,17,1,4.5,1305696483,4.209524,3.92093,0.290476,0.57907
...,...,...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,3.688556,2.50000,-1.188556,0.00000
100832,610,160527,4.5,1479544998,3.688556,4.50000,0.811444,0.00000
100833,610,160836,3.0,1493844794,3.688556,3.00000,-0.688556,0.00000
100834,610,163937,3.5,1493848789,3.688556,3.50000,-0.188556,0.00000


<font color=green> Теперь создадим новую таблицу и объединим в ней данные о рейтингах, жанрах и категориях.

In [ ]:
#ratings_mean = ratings.merge(ratings_u_mean, how = 'inner', left_on='userId', right_on='userId')

data1 = ratings_mean


In [ ]:
data1 = data1.merge(movies[['movieId','genres']], how = 'left', left_on='movieId', right_on='movieId')
data1

,userId,movieId,rating,timestamp,u_mean,m_mean,r_u,m_u,genres
0,1,1,4.0,964982703,4.366379,3.92093,-0.366379,0.07907,Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,3.636364,3.92093,0.363636,0.07907,Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,3.230263,3.92093,1.269737,0.57907,Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,3.448148,3.92093,-0.948148,-1.42093,Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,4.209524,3.92093,0.290476,0.57907,Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,3.688556,2.50000,-1.188556,0.00000,Action|Thriller
100832,610,160527,4.5,1479544998,3.688556,4.50000,0.811444,0.00000,Action|Crime|Drama
100833,610,160836,3.0,1493844794,3.688556,3.00000,-0.688556,0.00000,Action|Drama|Thriller
100834,610,163937,3.5,1493848789,3.688556,3.50000,-0.188556,0.00000,Horror|Thriller


In [ ]:
#df.groupby(['group_var'], as_index= False ).agg({'string_var ': ' '. join })
tags.groupby(['movieId'], as_index= False ).agg({'tag': '|'. join })

,movieId,tag
0,1,pixar|pixar|fun
1,2,fantasy|magic board game|Robin Williams|game
2,3,moldy|old
3,5,pregnancy|remake
4,7,remake
...,...,...
1567,183611,Comedy|funny|Rachel McAdams
1568,184471,adventure|Alicia Vikander|video game adaptation
1569,187593,Josh Brolin|Ryan Reynolds|sarcasm
1570,187595,Emilia Clarke|star wars


In [ ]:
data1 = data1.merge(tags.groupby(['movieId'], as_index= False ).agg({'tag': '|'. join })[['movieId','tag']], how = 'left', left_on='movieId', right_on='movieId')
data1

,userId,movieId,rating,timestamp,u_mean,m_mean,r_u,m_u,genres,tag
0,1,1,4.0,964982703,4.366379,3.92093,-0.366379,0.07907,Adventure|Animation|Children|Comedy|Fantasy,pixar|pixar|fun
1,5,1,4.0,847434962,3.636364,3.92093,0.363636,0.07907,Adventure|Animation|Children|Comedy|Fantasy,pixar|pixar|fun
2,7,1,4.5,1106635946,3.230263,3.92093,1.269737,0.57907,Adventure|Animation|Children|Comedy|Fantasy,pixar|pixar|fun
3,15,1,2.5,1510577970,3.448148,3.92093,-0.948148,-1.42093,Adventure|Animation|Children|Comedy|Fantasy,pixar|pixar|fun
4,17,1,4.5,1305696483,4.209524,3.92093,0.290476,0.57907,Adventure|Animation|Children|Comedy|Fantasy,pixar|pixar|fun
...,...,...,...,...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,3.688556,2.50000,-1.188556,0.00000,Action|Thriller,NaN
100832,610,160527,4.5,1479544998,3.688556,4.50000,0.811444,0.00000,Action|Crime|Drama,NaN
100833,610,160836,3.0,1493844794,3.688556,3.00000,-0.688556,0.00000,Action|Drama|Thriller,NaN
100834,610,163937,3.5,1493848789,3.688556,3.50000,-0.188556,0.00000,Horror|Thriller,NaN


<font color=green>  Отделим в промежуточную таблицу только нужные данные, что бы избавиться от балласта и избежать коллинеарности.

In [ ]:
data2 = data1[['userId', 'movieId', 'r_u', 'm_u']]
data2

,userId,movieId,r_u,m_u
0,1,1,-0.366379,0.07907
1,5,1,0.363636,0.07907
2,7,1,1.269737,0.57907
3,15,1,-0.948148,-1.42093
4,17,1,0.290476,0.57907
...,...,...,...,...
100831,610,160341,-1.188556,0.00000
100832,610,160527,0.811444,0.00000
100833,610,160836,-0.688556,0.00000
100834,610,163937,-0.188556,0.00000


<font color=green> Векторизуем категориальные данные о жанрах и тегах и добавим их в промежуточную таблицу.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def change_string_1(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [ ]:
str = "'Action|Adventure|Animation', 'Action|Animation|Comedy|Sci-Fi',"
change_string_1(str)

"'Action Adventure Animation','Action Animation Comedy SciFi',"

In [ ]:
#movies.genres.unique()

In [ ]:
movie_genres = [change_string_1(g) for g in data1.genres.values]

In [ ]:
len(movie_genres)

100836

In [ ]:
movie_genres[:10]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Animation Children Comedy Fantasy',
 'Adventure Animation Children Comedy Fantasy',
 'Adventure Animation Children Comedy Fantasy',
 'Adventure Animation Children Comedy Fantasy',
 'Adventure Animation Children Comedy Fantasy',
 'Adventure Animation Children Comedy Fantasy',
 'Adventure Animation Children Comedy Fantasy',
 'Adventure Animation Children Comedy Fantasy',
 'Adventure Animation Children Comedy Fantasy']

In [ ]:
data1.tag.unique()

array(['pixar|pixar|fun', 'moldy|old', nan, ...,
       'quirky|sweet|understated', 'paranoia|Suspenseful',
       'allegorical|uncomfortable|unsettling'], dtype=object)

In [ ]:
import six # правильно преобразовываем теги запсанные как float в строки
def change_string_3(q):
    if isinstance(q, six.string_types):
      s = q
      return ' '.join(s.replace(' ', '').replace('-', '').split('|'))
    else:
      s = repr(q)
      return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [ ]:
g = 3.0
change_string_3(g)

'3.0'

In [ ]:
movie_tags = [change_string_3(g) for g in data1.tag.values]

In [ ]:
len(movie_tags)

100836

In [ ]:
movie_tags[0:10]

['pixar pixar fun',
 'pixar pixar fun',
 'pixar pixar fun',
 'pixar pixar fun',
 'pixar pixar fun',
 'pixar pixar fun',
 'pixar pixar fun',
 'pixar pixar fun',
 'pixar pixar fun',
 'pixar pixar fun']

In [ ]:
tfidf_genres = TfidfVectorizer()
X_train_tfidf_genres = tfidf_genres.fit_transform(movie_genres)
X_train_tfidf_genres

<100836x20 sparse matrix of type '<class 'numpy.float64'>'
	with 274480 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_tags = TfidfVectorizer()
X_train_tfidf_tags = tfidf_tags.fit_transform(movie_tags)
X_train_tfidf_tags

<100836x1470 sparse matrix of type '<class 'numpy.float64'>'
	with 272042 stored elements in Compressed Sparse Row format>

In [ ]:
data31 = pd.DataFrame(X_train_tfidf_genres.toarray(), columns=tfidf_genres.get_feature_names_out())
data31

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.000000,0.363885,0.549735,0.508407,0.291944,0.000000,0.0,0.000000,0.470819,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.000000,0.363885,0.549735,0.508407,0.291944,0.000000,0.0,0.000000,0.470819,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.000000,0.363885,0.549735,0.508407,0.291944,0.000000,0.0,0.000000,0.470819,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.000000,0.363885,0.549735,0.508407,0.291944,0.000000,0.0,0.000000,0.470819,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.000000,0.363885,0.549735,0.508407,0.291944,0.000000,0.0,0.000000,0.470819,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,0.683829,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.729642,0.0,0.0
100832,0.545073,0.000000,0.000000,0.000000,0.000000,0.696269,0.0,0.467018,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
100833,0.590016,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.505525,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.629544,0.0,0.0
100834,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.840475,0.0,0.0,0.0,0.0,0.0,0.0,0.541851,0.0,0.0


In [ ]:
data32 = pd.DataFrame(X_train_tfidf_tags.toarray(), columns=tfidf_tags.get_feature_names_out())
data32

,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001like,2danimation,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100832,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<font color=green> Создадим исходную для модели таблицу данных объединив промежуточные.

In [ ]:
data4 = pd.concat([data2, data31, data32], axis=1, join='inner')
display(data4)

,userId,movieId,r_u,m_u,action,adventure,animation,children,comedy,crime,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,1,1,-0.366379,0.07907,0.000000,0.363885,0.549735,0.508407,0.291944,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,1,0.363636,0.07907,0.000000,0.363885,0.549735,0.508407,0.291944,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7,1,1.269737,0.57907,0.000000,0.363885,0.549735,0.508407,0.291944,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,1,-0.948148,-1.42093,0.000000,0.363885,0.549735,0.508407,0.291944,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,17,1,0.290476,0.57907,0.000000,0.363885,0.549735,0.508407,0.291944,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,160341,-1.188556,0.00000,0.683829,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100832,610,160527,0.811444,0.00000,0.545073,0.000000,0.000000,0.000000,0.000000,0.696269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100833,610,160836,-0.688556,0.00000,0.590016,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100834,610,163937,-0.188556,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<font color=green> Получилась очень большая и сильно разреженная таблица, хотя взяли сильно урезанный датасет.

In [ ]:
y = data4[['r_u']]
y.head()

,r_u
0,-0.366379
1,0.363636
2,1.269737
3,-0.948148
4,0.290476


In [ ]:
X = data4.drop('r_u', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train.head()

,r_u
80568,0.913151
50582,0.756098
8344,-0.499178
99603,0.342601
71701,-0.511259


<font color=green> Лучше всего отработала модель случайного леса.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
clf_rf = RandomForestRegressor(random_state=10)

In [ ]:
%%time
clf_rf.fit(X_train, y_train.values.ravel())

CPU times: user 9min 51s, sys: 1.5 s, total: 9min 53s
Wall time: 10min 15s


RandomForestRegressor(random_state=10)

In [ ]:
clf_rf.score(X_train, y_train)

0.9635312679269584

In [ ]:
clf_rf.score(X_test, y_test)

0.7441052861784259

In [ ]:
from sklearn.metrics import mean_squared_error


y_pred = clf_rf.predict(X_test)
y_train_pred = clf_rf.predict(X_train)
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_pred, squared=False))
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_pred, squared=False))

#rms = mean_squared_error(y_actual, y_predicted, squared=False)

Test RMSE = 0.4704
Train RMSE = 0.1787


<font color=green> Видим, что на тестовой выборке ошибка предсказания около половины балла. Т.к. в оценках объективно много субъективности, для целей рекомендации вряд ли требуется более точное предсказание.

In [ ]:
y_test.iloc[20167,:]

r_u   -0.565217
Name: 87803, dtype: float64

In [ ]:
y_pred[20167]

-0.7275945091544669

In [ ]:
y_test

,r_u
67037,0.284553
42175,-1.250580
93850,0.854028
6187,1.608043
12229,0.434783
...,...
57416,-0.147727
67290,-0.250580
33423,0.186047
98552,-2.347371


In [ ]:
X_test

,userId,movieId,m_u,action,adventure,animation,children,comedy,crime,documentary,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
67037,551,34162,0.491379,0.000000,0.000000,0.0,0.0,0.582902,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42175,232,59421,-0.750000,0.000000,0.000000,0.0,0.0,0.582902,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93850,288,8880,0.571429,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6187,414,1080,1.073034,0.000000,0.000000,0.0,0.0,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12229,577,2406,0.546296,0.468522,0.519279,0.0,0.0,0.416617,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57416,279,148626,-0.461538,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67290,232,46972,-0.239130,0.365012,0.000000,0.0,0.0,0.324574,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33423,456,802,0.663043,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98552,448,8137,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_test.movieId[X_test.movieId == 1958].index.tolist()

[87798, 87804, 87791, 87805, 87788, 87803]

<font color=green> Напомним сценарий.

<font color=green> Определим сценарий - клиент заходит в каталоге в конкретный жанр и ему выдается:
<br>
1 список из 7 самых популярных фильмов жанра на основании рейтингов фильмов
<br>
2 список из 7 самых популярных фильмов полученных после добавления к текущему жанру трех самых популярных жанра из истории клиента, и поиска по их комбинации методом ближайших соседей похожих фильмов с исключением уже просмотренных и рекомендованных по 1 списку и отсортированных по потенциальному рейтингу, полученному по модели случайного леса ( если истории еще нет, 3 дополнительных жанра выбираются случайным образом ).

<font color=green> Требуется построить модельку выбора фильмов по перечню жанров.

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 304.4+ KB


In [ ]:
ratings_m_mean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9724 entries, 0 to 9723
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9724 non-null   int64  
 1   m_mean   9724 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 152.1 KB


In [ ]:
movies = movies.merge(ratings_m_mean, how = 'inner', left_on='movieId', right_on='movieId')
movies

,movieId,title,genres,m_mean
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143
4,5,Father of the Bride Part II (1995),Comedy,3.071429
...,...,...,...,...
9719,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,4.000000
9720,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,3.500000
9721,193585,Flint (2017),Drama,3.500000
9722,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,3.500000


In [ ]:
movie_genres_ = [change_string_1(g) for g in movies.genres.values]
movie_genres_[:10]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller']

In [ ]:
tfidf_g = TfidfVectorizer()
X_train_tfidf = tfidf_g.fit_transform(movie_genres_)
X_train_tfidf

<9724x20 sparse matrix of type '<class 'numpy.float64'>'
	with 22046 stored elements in Compressed Sparse Row format>

In [ ]:
pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_g.get_feature_names_out())

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.000000,0.416817,0.516340,0.504733,0.267517,0.0,0.0,0.000000,0.483048,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.512358,0.000000,0.620425,0.000000,0.0,0.0,0.000000,0.593769,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.570532,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.821275,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.504636,0.0,0.0,0.466539,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.726418,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9719,0.435822,0.000000,0.614729,0.000000,0.318492,0.0,0.0,0.000000,0.575092,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9720,0.000000,0.000000,0.683007,0.000000,0.353868,0.0,0.0,0.000000,0.638967,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9721,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9722,0.578361,0.000000,0.815781,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
%%time
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=25, metric='euclidean') 
neigh.fit(X_train_tfidf)

CPU times: user 1.06 ms, sys: 0 ns, total: 1.06 ms
Wall time: 1.07 ms


NearestNeighbors(metric='euclidean', n_neighbors=25)

In [ ]:
test = change_string_1("Adventure|Comedy|Fantasy|Documentary")
#test = change_string_1("Comedy")

X_tfidf = tfidf_g.transform([test])

res = neigh.kneighbors(X_tfidf, return_distance=True)

In [ ]:
movies.iloc[res[1][0]].sort_values(by=['m_mean'], ascending=False)

,movieId,title,genres,m_mean
9699,188833,The Man Who Killed Don Quixote (2018),Adventure|Comedy|Fantasy,4.500000
862,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy,4.161765
7996,97757,'Hellboy': The Seeds of Creation (2004),Action|Adventure|Comedy|Documentary|Fantasy,4.000000
4844,7256,Touching the Void (2003),Adventure|Documentary,4.000000
5623,27368,Asterix & Obelix: Mission Cleopatra (Astérix &...,Adventure|Comedy|Fantasy,4.000000
6101,43289,"Bird People in China, The (Chûgoku no chôjin) ...",Adventure|Comedy|Drama|Fantasy,4.000000
3297,4467,"Adventures of Baron Munchausen, The (1988)",Adventure|Comedy|Fantasy,3.769231
2300,3052,Dogma (1999),Adventure|Comedy|Fantasy,3.645570
1597,2140,"Dark Crystal, The (1982)",Adventure|Fantasy,3.544118
3577,4911,Jabberwocky (1977),Adventure|Comedy|Fantasy,3.500000


<font color=green> 
Вспомогательная функция movie_gen_sort, выдающая список фильмов, отсортированный по среднему рейтингу фильма, на основании входящего перечня категорий. 
<br>
Список отобран согласно "близости" по модели ближайших соседей.


In [ ]:
def movie_gen_sort (kat):
  
  X_tfidf = tfidf_g.transform([kat])

  res = neigh.kneighbors(X_tfidf, return_distance=True)

  return movies.iloc[res[1][0]].sort_values(by=['m_mean'], ascending=False)

In [ ]:
res[1][0]

array([7996, 5821, 8143, 4844, 7579, 2604, 5623, 9699, 6706, 5614, 8343,
       3371, 7847,  862, 2300, 7479, 5724, 3297, 3577, 3571, 6101, 6249,
       1597, 4146, 2841])

In [ ]:
test = change_string_1("Adventure|Comedy|Fantasy|Documentary")
#test = change_string_1("Comedy")
movie_gen_sort(test)

,movieId,title,genres,m_mean
9699,188833,The Man Who Killed Don Quixote (2018),Adventure|Comedy|Fantasy,4.500000
862,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy,4.161765
7996,97757,'Hellboy': The Seeds of Creation (2004),Action|Adventure|Comedy|Documentary|Fantasy,4.000000
4844,7256,Touching the Void (2003),Adventure|Documentary,4.000000
5623,27368,Asterix & Obelix: Mission Cleopatra (Astérix &...,Adventure|Comedy|Fantasy,4.000000
6101,43289,"Bird People in China, The (Chûgoku no chôjin) ...",Adventure|Comedy|Drama|Fantasy,4.000000
3297,4467,"Adventures of Baron Munchausen, The (1988)",Adventure|Comedy|Fantasy,3.769231
2300,3052,Dogma (1999),Adventure|Comedy|Fantasy,3.645570
1597,2140,"Dark Crystal, The (1982)",Adventure|Fantasy,3.544118
3577,4911,Jabberwocky (1977),Adventure|Comedy|Fantasy,3.500000


In [ ]:
genres7 = movie_gen_sort(test)['title'].values.tolist()[0:7]
genres7

['The Man Who Killed Don Quixote (2018)',
 'Monty Python and the Holy Grail (1975)',
 "'Hellboy': The Seeds of Creation (2004)",
 'Touching the Void (2003)',
 'Asterix & Obelix: Mission Cleopatra (Astérix & Obélix: Mission Cléopâtre) (2002)',
 'Bird People in China, The (Chûgoku no chôjin) (1998)',
 'Adventures of Baron Munchausen, The (1988)']

<font color=green> Таким образом используя модель ближайших соседей, построенную на фильмах и жанрах можем получить отсортированный список заданной длины по жанру для 1 списка отдачи и по перечню жанров для 2 списка.

<font color=green> Требуется построить функцию, определяющую какие фильмы каких жанров пользователь уже посмотрел, и сортирующую по предполагаемому/(рассчитанному по модели) рейтингу

In [ ]:
user_ganres = data1[data1['userId']==577]['genres'].values.tolist()

In [ ]:
cat_ganres = ['Fantasy']
user_ganres[5]

'Comedy|Drama|Romance|War'

In [ ]:
from itertools import chain
from collections import Counter



In [ ]:
frequencies = Counter(chain.from_iterable(i.split('|') for i in user_ganres))
ls_gen = frequencies.most_common(4)
lst_genres = [change_string_1(g) for g,n in ls_gen]
lst_genres

['Drama', 'Action', 'Adventure', 'SciFi']

In [ ]:
def str_ganres_1(cat_ganres, user_ganres):
  top_4 = Counter(chain.from_iterable(i.split('|') for i in user_ganres)).most_common(4)
  return [change_string_1(g) for g,n in top_4]

In [ ]:
str_ganres_1(cat_ganres, user_ganres)

['Drama', 'Action', 'Adventure', 'SciFi']

<font color=green> 
Функция str_ganres выдает результат работы рекомендательной системы в виде двух списков:<br>
список cat_ganres7 - 7 лучших фильмов в категории, куда вошел пользователь, отсоритованных по рейтингу для фильма<br>
список rec_genres100 - все фильмы, отобранные на основании близости к 4 категориям, наиболее часто просматриваемым клиентом, и отсортированных по предсказанному для фильма рейтингу пользователя<br>
или, если у клиента нет истории, то *сгенерированным* случайно и отсоритованных по рейтингу для фильма<br>

In [ ]:
def str_ganres(cat_ganres, user_id, model):
  cat_ganres7 = movie_gen_sort(cat_ganres)['title'].values.tolist()[0:7]
  user_ganres = data1[data1['userId']==user_id]['genres'].values.tolist()
  user_history = 0
  user_history = len(user_ganres)
  ic(user_history)
  if user_history == 0:
    top_4 = random.sample(list(tfidf_g.get_feature_names_out()), 4)
  else:
    top_4 = Counter(chain.from_iterable(i.split('|') for i in user_ganres)).most_common(4)
    top_4 = [change_string_1(g) for g,n in top_4]
  if cat_ganres in top_4:
    top_4_str = '|'.join(top_4)
  else:
    top_4.insert (0, cat_ganres)
    top_4.pop(-1)
    top_4_str = '|'.join(top_4)
  ic(top_4_str)
  rec_genres100 = movie_gen_sort(top_4_str)['title'].values.tolist()
  #cat_ganres7 - 7 лучших фильмов в категории, куда вошел пользователь, отсоритованных по рейтингу для фильма
  #rec_genres100 - все фильмы, отобранные на основании близости к 4 категориям, наиболее просматриваемым клиентом,
  # или, если у клиента нет истории, сгенерированным случайно, отсоритованных по рейтингу для фильма
  if user_history == 0:
    return (cat_ganres7, rec_genres100[:7])
  else:
    rec_genres100 = rec_mov (user_id, cat_ganres7, rec_genres100, model)
    #ic(cat_ganres7)
    return (cat_ganres7, rec_genres100)

<font color=green> 
Вспомогательная функция rec_mov :
<br>
берем список фильмов, отобранный по 4 категориям, 
рассчитанный по модели "близости фильмов" методом ближайших соседей, без учета особенностей конкретного пользователя,
удаляем из него фильмы, которые пользователь уже посмотрел,
и возвращаем список рекомендованных фильмов, отсортированный по предполагаемым рейтингам пользователя,
рассчитанным по модели расчета предролагаемой рейтинговой оценки методом случайного леса.

In [ ]:
def rec_mov (user_id, cat_ganres7, rec_genres100, model):
  # вернем этот список   
  lst_rec_mov_sort = []

  y_pred_test = model.predict(X_test)
  y_pred_train = model.predict(X_train)

  X_model = pd.concat([X_test,X_train])
  y_pred_model = pd.concat([pd.DataFrame(y_pred_test),pd.DataFrame(y_pred_train)])
  
  ### Переводим для списков фильмов title -> movieID
  mov100_id = movies[movies['title'].isin(rec_genres100)][['movieId']]
  mov7_id = movies[movies['title'].isin(cat_ganres7)][['movieId']]

  ### просмотренные юзером из рекомендованных test train (удалить из mov)
  user_mov = X_model[(X_model['userId'] == user_id) & (X_model['movieId'].isin(mov100_id.movieId.tolist()))][['movieId']]

  ### buff -> выбрали все фильмы выбранных категорий из модели с пользовательскими рейтингами
  buff = X_model[(X_model['movieId'].isin(mov100_id.movieId.tolist()))][['movieId']]  

  lst_rec_mov = pd.DataFrame(buff.movieId.unique())
  ### оставили только уникальные фильмы
  lst_rec_mov = lst_rec_mov[~lst_rec_mov[0].isin(user_mov.movieId.tolist())]
  ### исключили фильмы, уже просмотренные пользователем
  lst_rec_mov = lst_rec_mov[~lst_rec_mov[0].isin(mov7_id.movieId.tolist())]
  ### исключили фильмы из списка выбранного по конкретному жанру
  lst_rec_mov.rename(columns={0: 'movieId'}, inplace=True)

  ### готовим данные для модели, для получения предсказания рейтингов
  X_lst_rec_mov = X_model[(X_model['movieId'].isin(lst_rec_mov.movieId.tolist()))]
  ### отобрали по списку матрицу входных данных 
  X_lst_rec_mov['index_1']=X_lst_rec_mov.index
  ### сохранили индекс в столбце
  X_index = X_lst_rec_mov.groupby(['movieId'])[['index_1']].agg(['max']).squeeze().tolist()
  ### сформировали список индексов, убрав повторения группировкой
  mf = X_model[X_model.index.isin(X_index)]
  ### сформировали матрицу признаков фильмов
  mf['userId'] = user_id
  ### поправили айдишник клиента на текущего пользователя
  y_pred_mf = clf_rf.predict(mf)
  ### получили предсказанные рейтинги
  y_pred_mf_r = pd.concat([mf[['movieId']].reset_index(drop=True),pd.DataFrame(y_pred_mf).reset_index(drop=True)], axis=1)
   ### получили неотсортированныйи необрезанный датафрейм предсказанных рейтингов
  y_pred_mf_r.rename(columns={0: 'r'}, inplace=True)
  lst_rec_mov_s = y_pred_mf_r.sort_values(by=['r'], ascending=False)['movieId'].tolist()
  lst_rec_mov_sort = movies[movies['movieId'].isin(lst_rec_mov_s[:7])][['title']]
  return (lst_rec_mov_sort.title.tolist())

<font color=green> Тестовые данные.

In [ ]:
cat_ganres = 'Fantasy'
#cat_ganres = 'Adventure'
user_id = 577
#user_id = 111577

In [ ]:
ganr = []
rec = []

In [ ]:
ganr, rec = str_ganres(cat_ganres, user_id, clf_rf)


ic| user_history: 161
ic| top_4_str: 'Fantasy|Drama|Action|Adventure'
<ipython-input-91-498b86e6d679>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_lst_rec_mov['index_1']=X_lst_rec_mov.index
<ipython-input-91-498b86e6d679>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mf['userId'] = user_id


In [ ]:
ganr

['Andalusian Dog, An (Chien andalou, Un) (1929)',
 "Akira Kurosawa's Dreams (Dreams) (1990)",
 'After Life (Wandafuru raifu) (1998)',
 '8 1/2 (8½) (1963)',
 'Hereafter (2010)',
 'Fast Runner, The (Atanarjuat) (2001)',
 'Imaginarium of Doctor Parnassus, The (2009)']

In [ ]:
rec

['Dragonheart (1996)',
 'Conan the Barbarian (1982)',
 'Willow (1988)',
 'Supergirl (1984)',
 'Highlander: Endgame (Highlander IV) (2000)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Tomb Raider (2018)']

<font color=green> 
Гибридная поисковая система работает.
<br>
К достоинствам можно отнести почти гладкий холодный старт для клиента.
<br>
При отсутствии истории отдаются рекомендации на основе близости по категориям и общей статистики по рейтингам.
<br>
Чем больше накапливается статистики у клиента, тем точнее становятся рекомендации во втором списке, учитывающем его предпочтения.
<br>